In [1]:
import scipy.sparse
import json
import string
import pymorphy2

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LinearRegression
from sklearn.feature_extraction.text import TfidfVectorizer

from tqdm import tqdm_notebook
from multiprocessing import Pool, cpu_count

In [2]:
def process_line(line):
    item = json.loads(line)
    del item['image'], item['content']
    item['title'] = item['title']
    return item

In [3]:
items_count = 0
with open('../data/interim/items.json') as items_file:
    for line in items_file:
        items_count += 1
items_count

328050

In [4]:
items = []
with open('../data/interim/items.json') as items_file:
    with Pool(cpu_count()) as p:
        items = list(tqdm_notebook(p.imap(process_line, items_file), total=items_count))
items = pd.DataFrame(items)
items.head()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  after removing the cwd from sys.path.


,itemId,title
0,0,Пять забавных «морщинистых» пород собак
1,1,История улицы Ирининской в Гомеле
2,2,Зачем Дудь всё время спрашивает гостей програм...
3,3,Светлана Зейналова крестила младшую дочь
4,4,"ГКНБ: бандиты в ГБАО делают вид, что рассталис..."


In [2]:
import gc
gc.collect()

7

In [3]:
data = []
row = []
col = []

train_lines = sum(1 for line in open('../data/interim/train.json','r'))

with open('../data/interim/train.json') as train_file:
    for i, line in enumerate(tqdm_notebook(train_file, total=train_lines)):
        json_line = json.loads(line)
        for item, rating in json_line['trainRatings'].items():
            data.append((-1) ** (int(rating) + 1))
            row.append(i)
            col.append(int(item))
train = pd.DataFrame({'userId': row, 'itemId': col, 'rating': data})
del data, row, col
train.userId.nunique()
# train_int = scipy.sparse.coo_matrix((data, (row, col)))

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:8: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  


42977

In [10]:
items = items.set_index('itemId')

In [13]:
items.iloc[train[(train.userId == 0) & (train.rating == 1)].itemId][10:20]

,title
itemId,
184990,В России собираются ввести налог на курение
75612,Отдых на халяву? Легко!
167043,"10 пар слов, значения которых мы всегда путаем"
204085,Тест с опущенными в воду руками на полминуты с...
236149,Что американцам кажется смешным. ТОП-5
306948,Недавно объявилась моя бывшая свекровь.
254293,"""Класть"" или ""ложить"": как правильно?"
154292,Прокрустовы туфельки
128617,"3 наречия, в которых делают ошибки безграмотны..."


In [15]:
items = pd.read_csv('../data/processed/processed_items.csv', index_col='itemId')

In [16]:
items['title'] = items['title'].str.split()
titles = list(items['title'].values)

In [6]:
test = pd.read_csv('../data/external/solution.csv')
test.groupby('userId')['rating'].sum()

userId
1         68
2         26
54       104
69        86
77        27
        ... 
42933    162
42961     56
42971     63
42972    132
42975     17
Name: rating, Length: 4349, dtype: int64

In [7]:
test.groupby('userId')['rating'].count()

userId
1         596
2         349
54       1727
69        734
77        103
         ... 
42933     670
42961     604
42971     424
42972    1007
42975     407
Name: rating, Length: 4349, dtype: int64

In [3]:
processed = pd.read_csv('../data/processed/processed_items.csv')
processed.head()

,itemId,content,image,title
0,0,согласиться дорогой любитель собака до что же ...,[-0.169 0.129 0.067 0.019 0.281 -0.245 0....,пять забавный морщинистый порода собака
1,1,контур три поперечный улица состоять до недавн...,[-0.158 -0.112 -0.325 0.05 -0.114 0.002 -0....,история улица ирининский в гомель
2,2,источник http infodays ru вообще он как то сам...,[ 0.084 -0.181 0.008 0.34 -0.03 -0.197 -0....,зачем дудь весь время спрашивать гость програм...
3,3,41 летний светлана зейналов решить окрестить 5...,[ 0.034 -0.119 -0.062 0.025 0.128 -0.041 0....,светлана зейналов крестить младший дочь
4,4,организовать преступный группировка гбао делат...,[-0.061 -0.015 -0.198 -0.047 0.054 0.029 -0....,гкнб бандит в гбао делать вид что расстаться с...


In [9]:
' '.join(list(map(tag, processed.title[0].split())))

'пять_NUM забавный_ADJ морщинистый_ADJ порода_NOUN собака_NOUN'

In [6]:
from pymystem3 import Mystem

conversion_table = {
    'A': 'ADJ',
    'ADV': 'ADV',
    'ADVPRO': 'ADV',
    'ANUM': 'ADJ',
    'APRO': 'DET',
    'COM': 'ADJ',
    'CONJ': 'SCONJ',
    'INTJ': 'INTJ',
    'NONLEX': 'X',
    'NUM': 'NUM',
    'PART': 'PART',
    'PR': 'ADP',
    'S': 'NOUN',
    'SPRO': 'PRON',
    'UNKN': 'X',
    'V': 'VERB'
}

m = Mystem()

def tag(word='пожар'):
    processed = m.analyze(word)[0]
    if 'analysis' not in processed or not processed["analysis"]:
        return None
    lemma = processed["analysis"][0]["lex"].lower().strip()
    pos = processed["analysis"][0]["gr"].split(',')[0]
    pos = pos.split('=')[0].strip()
    pos = conversion_table.get(pos)
    tagged = lemma + '_' + pos
    return tagged